In [2]:
import pandas as pd
import numpy as np
import os

def load_data(datapath):
    csv_path = os.path.abspath(datapath)
    return pd.read_csv(csv_path, header=None)

def data_split(dataset):
    train = dataset.sample(frac=0.8)
    val = dataset.drop(train.index)
    train_y = train.iloc[:,0]
    train_x = train.drop(0, axis=1)
    val_y = val.iloc[:,0]
    val_x = val.drop(0, axis=1)
    return train_x, train_y, val_x, val_y

def main(datapath):
    train_x, train_y, val_x, val_y = data_split(load_data(datapath))
    myMLP = MLP(4,13,3,0.001)
    prediction = myMLP.predict(train_x.iloc[0])
    return(prediction)


class MLP:

    def __init__(self, n_hidden, n_input_feat, n_classes, learning_rate):
        self.n_hidden = n_hidden #4
        self.n_input = n_input_feat #13
        self.n_output = n_classes #3
        self.lr = learning_rate #0.0001

        self.v = np.random.uniform(size=((self.n_input+1)*self.n_hidden,1))
        self.w = np.random.uniform(size=((self.n_hidden+1)*self.n_output,1))


    def hidden_layer(self, input, v):
        input = np.array(input).reshape(self.n_input,1)
        x = np.append(np.array([[1]]), input, axis=0)
        #x = vector of length 14
        #v = vector of length 56
        a = np.zeros(shape=(self.n_hidden,1))
        #a = 0 vector of length 4
        for node in range(self.n_hidden): #0,1,2,3
            a_x = np.zeros(shape=(self.n_input+1,1)) 
            #a_x = 0 vector of length 14
            for index in range(len(v)): #0,1,2,3,...,59,60
                if index % self.n_hidden == node:
                    a_x[index//self.n_hidden] = v[index]
            a[node] = np.dot(a_x.T,x)
        #ReLU
        for entry in range(len(a)):
            a[entry] = (lambda z: z*(z > 0))(a[entry])
        return a
    
    def output_layer(self, a, w):
        a = np.array(a).reshape(self.n_hidden,1)
        a = np.append(np.array([[1]]), a, axis=0)
        print(a)
        #a = vector of length 5
        #w = vector of length 15
        y = np.zeros(shape=(self.n_output,1))
        #y = 0 vector of length 3
        for node in range(self.n_output): #0,1,2
            y_a = np.zeros(shape=(self.n_hidden+1,1)) 
            #y_a = 0 vector of length 5
            for index in range(len(w)): #0,1,2,3,...,14,15
                if index % self.n_output == node:
                    y_a[index//self.n_output] = w[index]
            y[node] = np.dot(y_a.T,a)
        #softmax
        print(y)
        y = np.exp(y)
        print(y.exp)
        sum = np.sum(y)
        y = y*(1/sum)
        for entry in range(len(y)):
            y[entry] = 1 if y[entry] == max(y) else 0
        return y

    def predict(self, input):
        a = self.hidden_layer(input, self.v)
        y = self.output_layer(a, self.w)
        return y




main('wine.data')

[[  1.        ]
 [292.2632315 ]
 [357.37049509]
 [305.7502975 ]
 [ 54.19589904]]
[[675.52262052]
 [534.76115984]
 [369.51057903]]


AttributeError: 'numpy.ndarray' object has no attribute 'exp'

In [ ]:
# import numpy as np
# y = np.random.uniform(size=(5,1))
# print(y)
# y = np.exp(y)
# sum = np.sum(y)
# y = y*(1/sum)
# for entry in range(len(y)):
#     y[entry] = 1 if y[entry] == max(y) else 0
# print(y)

[[0.5123662 ]
 [0.7967629 ]
 [0.67040199]
 [0.6005905 ]
 [0.7216754 ]]
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]]
